In [1]:
#python==3.9.13
#pandas==1.4.4
#brightway2==2.4.3
#ecoinvent version==ecoinvent 3.9_cutoff

In [2]:
import pandas as pd   #used for dealing with Excel files and creating data format
import brightway2 as bw   #used for LCA calculation
from brightway2 import *   
import bw2analyzer as bwa   #used for the LCA contribution analysis
from bw2data.parameters import ActivityParameter, DatabaseParameter, ProjectParameter, Group

<span style="color:black;font-weight:600;font-size:20px">
    1. Create a project
</span>

In [3]:
#List the available projects
list(bw.projects)

[Project: default,
 Project: BW2 introduction,
 Project: B4B18,
 Project: 123,
 Project: NRC2,
 Project: NRC3,
 Project: NRC4]

In [4]:
#create or access a new project
bw.projects.set_current("NRC4")   #the project name need to be determined (e.g., NRC)
bw.bw2setup()   #import the biosphere3 database, LCIA methods and some other data

Biosphere database already present!!! No setup is needed


In [5]:
#check our created project
list(bw.projects)

[Project: default,
 Project: BW2 introduction,
 Project: B4B18,
 Project: 123,
 Project: NRC2,
 Project: NRC3,
 Project: NRC4]

<span style="color:black;font-weight:600;font-size:20px">
    2. Database import
</span>

In [6]:
#list the available databases
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.9_cutoff
	elegancy

In [7]:
#import ecoinvent database
#note: the variable name need to be updated according to the version of ecoinvent database (e.g., ecoinvent 3.9_cutoff, fpei39cut, ei39cut) 

if 'ecoinvent 3.9_cutoff' in bw.databases:   
    print("Database has already been imported.")
else:
    # mind that the ecoinvent file must be unzipped; then: path to the datasets subfolder
    fpei39cut = r"C:\Users\89751\OneDrive\桌面\Brightway2\Database\ecoinvent 3.9_cutoff_ecoSpold02\datasets"  
    ei39cut = bw.SingleOutputEcospold2Importer(fpei39cut, 'ecoinvent 3.9_cutoff')   
    ei39cut
    ei39cut.apply_strategies()
    ei39cut.statistics()

Database has already been imported.


In [8]:
#write database to disk --> actually saves it
ei39cut.write_database()

NameError: name 'ei39cut' is not defined

In [9]:
#list the available databases - again
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.9_cutoff
	elegancy

<span style="color:black;font-weight:600;font-size:20px">
    3. LCA
</span>

In [10]:
#assign a variable to the database (eidb, bs)
eidb = bw.Database('ecoinvent 3.9_cutoff')
bs = bw.Database('biosphere3')     

In [11]:
#create a new database (e.g., elegancy) to store created activities
db1 = bw.Database('elegancy')
db1.register()

In [12]:
#list the available databases - again
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.9_cutoff
	elegancy

<span style="color:black;font-weight:600;font-size:16px">
Define functions
<span>

In [20]:
#define a function to create new activity in the database
def create_new_activity (name, unit, uuid):
    activity = db1.new_activity(code=uuid, name=name, unit=unit)   
    activity.save() #save the created activity

In [14]:
#define a function to search the activities (unit processes) from an ecoinvent database
def search_ei_act(excel_file_path, sheet_name):
    act_identified_dict = {'Name':[], 'location':[], 'unit':[], 'code':[]}
    
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name, engine='openpyxl')
    
    for _, row in df.iterrows():
        if pd.isna(row[0]):
            continue
            
        name = row[0]
        location = row[1]
        
        matching_activities = [act for act in eidb if name in act['name'] and location in act['location']]
        
        for act in matching_activities:
            act_identified_dict['Name'].append(act['name'])
            act_identified_dict['location'].append(act['location'])
            act_identified_dict['unit'].append(act['unit'])
            act_identified_dict['code'].append(act['code'])
            
        act_identified_df = pd.DataFrame(act_identified_dict)
    return act_identified_df
    

In [15]:
#define a function to broad search the activities (unit processes) from an ecoinvent database (without location)
def broad_search_ei_act(excel_file_path):
    act_identified_dict = {'Name':[], 'location':[], 'unit':[], 'code':[]}
    
    df = pd.read_excel(excel_file_path, sheet_name="Activity_without_location", engine='openpyxl')
    
    for _, row in df.iterrows():
            
        name = row[0]
        
        matching_activities = [act for act in eidb if name in act['name']]
        
        for act in matching_activities:
            act_identified_dict['Name'].append(act['name'])
            act_identified_dict['location'].append(act['location'])
            act_identified_dict['unit'].append(act['unit'])
            act_identified_dict['code'].append(act['code'])
            
        act_identified_df = pd.DataFrame(act_identified_dict)
    return act_identified_df

In [16]:
#define a function to search elementary flows from the biosphere3 database
def search_bs_act(excel_file_path, sheet_name):
    act_identified_dict = {'Name':[], 'unit':[], 'code':[]}
    
    bs = bw.Database('biosphere3')
    
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name, engine='openpyxl')
    
    for _, row in df.iterrows():
        if pd.isna(row[9]):
            continue
            
        name = str(row[9])
        
        matching_activities = [act for act in bs if name in act['name']]
        
        for act in matching_activities:
            act_identified_dict['Name'].append(act['name'])
            act_identified_dict['unit'].append(act['unit'])
            act_identified_dict['code'].append(act['code'])
            
        act_identified_df = pd.DataFrame(act_identified_dict)
    return act_identified_df

In [17]:
#define a function to input technosphere and elementary flows into the activity
def input_flows (excel_file_path, sheet_name, activity_uuid):
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
    
    activity = db1.get(activity_uuid)
    for idx, row in df.iterrows():
        if not pd.isna(row['uuid1']):
            uuid = row['uuid1']
            input_activity = eidb.get(uuid)        
            activity.new_exchange(input=input_activity.key, amount=row['amount1'], unit=row['unit1'], type='technosphere').save()   
            activity.save()   
        
    for idx, row in df.iterrows():
        if not pd.isna(row['uuid2']):
            uuid = row['uuid2']
            input_activity = bs.get(uuid)
            activity.new_exchange(input=input_activity.key, amount=row['amount2'], unit=row['unit2'], type='biosphere').save()   
            activity.save()  

In [18]:
#define a function to search LCI methods: 
def search_LCIA(excel_file_path):
    lcia_methods = []
       
    df = pd.read_excel(excel_file_path, sheet_name="LCIA_methods", engine='openpyxl')  
    
    for bw_method in methods:
        for _, row in df.iterrows():
            lvl_0 = row[0]
            lvl_1 = row[1]
            lvl_2 = row[2]
            if lvl_0 in bw_method[0]:
                if lvl_1 in bw_method[1]:
                    if lvl_2 in bw_method[2]:
                        lcia_methods.append(bw_method)
                    else: continue
                else: continue
            else: continue
                
    return lcia_methods

<span style="color:black;font-weight:600;font-size:16px">
Build foreground system
<span>

In [21]:
#import the excel file
excel_file_path = 'C:/Users/89751/OneDrive/桌面/Brightway2/test11.xlsx'   #update the excel address here

In [22]:
#create activities (name, unit, and uuid)
Activity1_name = 'Activity1'   #Activity_name
Activity1_unit = 'kg'    #Activity_unit
Activity1_uuid = '1111'    #Activity_uuid

Activity2_name = 'Activity2'
Activity2_unit = 'kg'
Activity2_uuid = '2222'

Activity3_name = 'Activity3'
Activity3_unit = 'kg'
Activity3_uuid = '3333'


#create activities in the database ('elegency')
#call: create_new_activity(Activity_name, Activity_unit, Activity_uuid)
Activity1 = create_new_activity (Activity1_name, Activity1_unit, Activity1_uuid)
Activity2 = create_new_activity (Activity2_name, Activity2_unit, Activity2_uuid)
Activity3 = create_new_activity (Activity3_name, Activity3_unit, Activity3_uuid)

In [23]:
#search unit processes from the database
sheet_name = 'Activity1'   #update the sheet_name

#call: search_ei/bs_act(excel_file_path, sheet_name)
techno_flow_options = search_ei_act(excel_file_path, sheet_name)   #search from the ecoinvent database
element_flow_options = search_bs_act(excel_file_path, sheet_name)   #search from the biosphere database

In [24]:
#check identified technosphere flows
techno_flow_options

,Name,location,unit,code
0,"market for battery cell, Li-ion, LiMn2O4",GLO,kilogram,4834ba6a5ffd0c3cc22cbda7c95e6a53
1,"market for cable, data cable in infrastructure",GLO,meter,a258b4b6cb7d325096324358813d8d60
2,"market for cable, three-conductor cable",GLO,meter,a956b9992ddea711a0da44c0c9f6eea5
3,market for metal working factory,GLO,unit,6151193e6f9d2a12d8628c206bcfeeda
4,"market for printed wiring board, surface mount...",GLO,kilogram,7471771e14369976fb8c588cbf029c3d
5,"market for printed wiring board, surface mount...",GLO,kilogram,23beebbf3e563b5b2be135f91d60788d
6,market for reinforcing steel,GLO,kilogram,e9449b977358d4978e600de7c8d63bc7
7,"market for sheet rolling, steel",GLO,kilogram,ff2b3d0719d526d234a98971e4de530d


In [25]:
#check identified elementary flows
element_flow_options

,Name,unit,code
0,"Water, well, in ground",cubic meter,67c40aae-d403-464d-9649-c12695e43ad8


In [26]:
#If you cannot find appropriate technosphere flows, input the flow name in sheet 'Activity_without_location' and try the broad search_ei_act(excel_file_path) function

techno_flow_broad_option = broad_search_ei_act(excel_file_path) 
techno_flow_broad_option

,Name,location,unit,code
0,"market for battery cell, Li-ion, LiMn2O4",GLO,kilogram,4834ba6a5ffd0c3cc22cbda7c95e6a53
1,"market for cable, data cable in infrastructure",GLO,meter,a258b4b6cb7d325096324358813d8d60


In [27]:
#next, select the relevant flows and enter their UUID codes into Excel: uuid1 (technosphere) and uuid2 (biosphere).
#repeat this process for each activity sheet (e.g., Activity1, Activity2, Activity3) until all flows are matched with their respective UUIDs.

In [28]:
#input technosphere and elementary flows into the activity1

sheet_name = 'Activity1'   #update the sheet_name here
Activity_uuid = Activity1_uuid   #update the Activity_uuid here

#call: input_flows(excel_file_path, sheet_name, Activity_uuid)  
input_flows(excel_file_path, sheet_name, Activity_uuid)   

activity = db1.get(Activity_uuid)   #print all exchanges in the activity1
for exc in activity.exchanges(): 
    print(exc)

Exchange: 0.799 kilogram 'market for battery cell, Li-ion, LiMn2O4' (kilogram, GLO, None) to 'Activity1' (kg, GLO, None)>
Exchange: 0.373 meter 'market for cable, data cable in infrastructure' (meter, GLO, None) to 'Activity1' (kg, GLO, None)>
Exchange: 0.025 meter 'market for cable, three-conductor cable' (meter, GLO, None) to 'Activity1' (kg, GLO, None)>
Exchange: 4.58e-10 unit 'market for metal working factory' (unit, GLO, None) to 'Activity1' (kg, GLO, None)>
Exchange: 0.00102 kilogram 'market for printed wiring board, surface mounted, unspecified, Pb containing' (kilogram, GLO, None) to 'Activity1' (kg, GLO, None)>
Exchange: 0.00237 kilogram 'market for printed wiring board, surface mounted, unspecified, Pb free' (kilogram, GLO, None) to 'Activity1' (kg, GLO, None)>
Exchange: 0.145 kilogram 'market for reinforcing steel' (kilogram, GLO, None) to 'Activity1' (kg, GLO, None)>
Exchange: 0.145 kilogram 'market for sheet rolling, steel' (kilogram, GLO, None) to 'Activity1' (kg, GLO, No

In [29]:
#input technosphere and elementary flows into the activity2

sheet_name = 'Activity2'   #update the sheet_name here
Activity_uuid = Activity2_uuid   #update the Activity_uuid here

#call: input_flows(excel_file_path, sheet_name, Activity_uuid)  
input_flows(excel_file_path, sheet_name, Activity_uuid)   

activity = db1.get(Activity_uuid)   #print all exchanges in the activity2
for exc in activity.exchanges(): 
    print(exc)

Exchange: 0.108 kilowatt hour 'market group for electricity, low voltage' (kilowatt hour, GLO, None) to 'Activity2' (kg, GLO, None)>
Exchange: 2.93e-05 cubic meter 'Water, well, in ground' (cubic meter, None, ('natural resource', 'in water')) to 'Activity2' (kg, GLO, None)>


In [30]:
#input technosphere and elementary flows into the activity3

sheet_name = 'Activity3'   #update the sheet_name here
Activity_uuid = Activity3_uuid   #update the Activity_uuid here

#call: input_flows(excel_file_path, sheet_name, Activity_uuid)  
input_flows(excel_file_path, sheet_name, Activity_uuid)   

activity = db1.get(Activity_uuid)   #print all exchanges in the activity3
for exc in activity.exchanges(): 
    print(exc)

Exchange: 0.108 kilowatt hour 'market group for electricity, low voltage' (kilowatt hour, GLO, None) to 'Activity3' (kg, GLO, None)>
Exchange: 0.373 meter 'market for cable, data cable in infrastructure' (meter, GLO, None) to 'Activity3' (kg, GLO, None)>
Exchange: 2.93e-05 cubic meter 'Water, well, in ground' (cubic meter, None, ('natural resource', 'in water')) to 'Activity3' (kg, GLO, None)>


<span style="color:black;font-weight:600;font-size:16px">
LCIA
<span>

In [31]:
#list all of the selected LCIA methods
method_key = search_LCIA(excel_file_path) 
method_key

[('TRACI v2.1 no LT',
  'acidification no LT',
  'acidification potential (AP) no LT'),
 ('TRACI v2.1 no LT',
  'climate change no LT',
  'global warming potential (GWP100) no LT'),
 ('TRACI v2.1 no LT',
  'ecotoxicity: freshwater no LT',
  'ecotoxicity: freshwater no LT'),
 ('TRACI v2.1 no LT',
  'eutrophication no LT',
  'eutrophication potential no LT'),
 ('TRACI v2.1 no LT',
  'ozone depletion no LT',
  'ozone depletion potential (ODP) no LT'),
 ('TRACI v2.1', 'acidification', 'acidification potential (AP)'),
 ('TRACI v2.1', 'climate change', 'global warming potential (GWP100)'),
 ('TRACI v2.1', 'ecotoxicity: freshwater', 'ecotoxicity: freshwater'),
 ('TRACI v2.1', 'eutrophication', 'eutrophication potential'),
 ('TRACI v2.1', 'ozone depletion', 'ozone depletion potential (ODP)')]

In [32]:
#drop all unwanted LCIA methods from the list (e.g., 'TRACI v2.1 no LT')
filtered_key = [item for item in method_key if item[0] != 'TRACI v2.1 no LT']   #replace the keyword here (e.g., 'TRACI v2.1 no LT')
filtered_key

[('TRACI v2.1', 'acidification', 'acidification potential (AP)'),
 ('TRACI v2.1', 'climate change', 'global warming potential (GWP100)'),
 ('TRACI v2.1', 'ecotoxicity: freshwater', 'ecotoxicity: freshwater'),
 ('TRACI v2.1', 'eutrophication', 'eutrophication potential'),
 ('TRACI v2.1', 'ozone depletion', 'ozone depletion potential (ODP)')]

<span style="color:black;font-weight:600;font-size:16px">
LCA calculation for multiple activities
<span>

In [51]:
#define functional units for each activity

activity1 = db1.get(Activity1_uuid)   #update the activity_uuid if need
activity2 = db1.get(Activity2_uuid)
activity3 = db1.get(Activity3_uuid)

functional_units = [{activity1: 1}, {activity2: 1}, {activity3: 1}]   #update the reference flow amounts (e.g., 'activity1: 1' means manufatcuring 1 unit of activity1)

In [87]:
# Prepare DataFrame to hold results
results_df = pd.DataFrame(columns=['Activity', 'Impact Category', 'Score', 'Unit'])

rows_list = []

# LCA result calculation
for functional_unit in functional_units:
    process_name = list(functional_unit.keys())[0]['name']
    for LCA_method in filtered_key:
        lca = bw.LCA(functional_unit, LCA_method)
        lca.lci()
        lca.lcia()
        score = lca.score
        unit = bw.methods[LCA_method]['unit']
        
        row_dict = {
            'Activity': process_name,
            'Impact Category': LCA_method[2],
            'Score': lca.score,
            'Unit': bw.methods[LCA_method]['unit']  
        }
        
        # Append the row dictionary to the rows list
        rows_list.append(row_dict)

Overall_LCA_result = pd.DataFrame(rows_list)

print(Overall_LCA_result)

     Activity                    Impact Category         Score          Unit
0   Activity1       acidification potential (AP)  6.943708e-02     kg SO2-Eq
1   Activity1  global warming potential (GWP100)  8.378657e+00     kg CO2-Eq
2   Activity1            ecotoxicity: freshwater  6.103306e+02          CTUe
3   Activity1           eutrophication potential  4.139928e-02       kg N-Eq
4   Activity1    ozone depletion potential (ODP)  2.915551e-07  kg CFC-11-Eq
5   Activity2       acidification potential (AP)  3.483250e-04     kg SO2-Eq
6   Activity2  global warming potential (GWP100)  7.966781e-02     kg CO2-Eq
7   Activity2            ecotoxicity: freshwater  1.567089e+00          CTUe
8   Activity2           eutrophication potential  2.893012e-04       kg N-Eq
9   Activity2    ozone depletion potential (ODP)  5.885274e-10  kg CFC-11-Eq
10  Activity3       acidification potential (AP)  3.234964e-03     kg SO2-Eq
11  Activity3  global warming potential (GWP100)  1.819181e-01     kg CO2-Eq

In [89]:
#reorganize the result format
Overall_LCA_result1 = Overall_LCA_result.pivot(index=['Impact Category','Unit'], columns='Activity', values='Score')
Overall_LCA_result1

,Activity,Activity1,Activity2,Activity3
Impact Category,Unit,,,
acidification potential (AP),kg SO2-Eq,6.943708e-02,3.483250e-04,3.234964e-03
ecotoxicity: freshwater,CTUe,6.103306e+02,1.567089e+00,7.112185e+01
eutrophication potential,kg N-Eq,4.139928e-02,2.893012e-04,2.376957e-03
global warming potential (GWP100),kg CO2-Eq,8.378657e+00,7.966781e-02,1.819181e-01
ozone depletion potential (ODP),kg CFC-11-Eq,2.915551e-07,5.885274e-10,1.725403e-09


In [90]:
#export LCA results (multiple activities) to an excel file

excel_file_path = 'C:/Users/89751/OneDrive/桌面/Book1.xlsx'   #create a new Excel file and load the address here

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    # Write the first DataFrame (df) to a sheet named 'lca_result'
    Overall_LCA_result.to_excel(writer, sheet_name='overall_lca_result', index=False)

<span style="color:black;font-weight:600;font-size:16px">
LCA calculation for the single activity with contribution analysis
<span>

In [80]:
#define functional units for the activity

activity1 = db1.get(Activity1_uuid)   #update the activity_uuid if need
functional_unit = {activity1:1} 

In [81]:
#LCA results calculation and contribution analysis
lca_results = []
lca_contribution_results = []

i=5   #Set the the number of flows shown from the contribution analysis.

for LCA_method in filtered_key:
    lca = bw.LCA(functional_unit, LCA_method)
    lca.lci()
    lca.lcia()
    score = lca.score
    ca = bwa.ContributionAnalysis()
    data = ca.annotated_top_processes(lca, limit=i)
    lca_results.append(score)
    lca_contribution_results.append(data)
    
print (lca_results)

[0.06943708440592876, 8.378656867679991, 610.3306300717932, 0.041399280409140676, 2.915551052578594e-07]


In [82]:
lca.demand

{'Activity1' (kg, GLO, None): 1}

In [83]:
lca_results

[0.06943708440592876,
 8.378656867679991,
 610.3306300717932,
 0.041399280409140676,
 2.915551052578594e-07]

In [84]:
#print the results
for result, method in zip(lca_results, filtered_key):
    key = tuple(method)
    method_key = method[2]
    process_name = list(functional_unit.keys())[0]['name']
    unit = bw.methods.get(key).get('unit', 'unknown unit')  # Provide a default if 'unit' is not found
    print("The {} results for the {} process is {:.3e} {}.".format(method_key, process_name, result, unit))

The acidification potential (AP) results for the Activity1 process is 6.944e-02 kg SO2-Eq.
The global warming potential (GWP100) results for the Activity1 process is 8.379e+00 kg CO2-Eq.
The ecotoxicity: freshwater results for the Activity1 process is 6.103e+02 CTUe.
The eutrophication potential results for the Activity1 process is 4.140e-02 kg N-Eq.
The ozone depletion potential (ODP) results for the Activity1 process is 2.916e-07 kg CFC-11-Eq.


In [100]:
#print the result as a pd.dataframe format

df = pd.DataFrame(columns=['method_key', 'result', 'unit'])
dfs_to_concat = []


for result, method in zip(lca_results, filtered_key):
    key = tuple(method)
    method_key, unit = method[2], bw.methods.get(key, {'unit': 'unknown unit'}).get('unit')
    
    result = pd.to_numeric(result, errors='coerce')
    
    row = {'method_key': method_key, 'result': result, 'unit': unit}
    dfs_to_concat.append(pd.DataFrame([row]))

LCA_result1 = pd.concat(dfs_to_concat, ignore_index=True)

print(LCA_result)

                          method_key        result          unit
0       acidification potential (AP)  6.943708e-02     kg SO2-Eq
1  global warming potential (GWP100)  8.378657e+00     kg CO2-Eq
2            ecotoxicity: freshwater  6.103306e+02          CTUe
3           eutrophication potential  4.139928e-02       kg N-Eq
4    ozone depletion potential (ODP)  2.915551e-07  kg CFC-11-Eq


In [101]:
#show the contribution analysis results
lca_contribution_results

[[(0.02013515645314219,
   0.28637685360579285,
   'synthetic graphite production, battery grade' (kilogram, RoW, None)),
  (0.0028187457240390736,
   0.005609730377687335,
   'smelting of copper concentrate, sulfide ore' (kilogram, CN, None)),
  (0.002521854817473607,
   0.005500447441776105,
   'smelting of copper concentrate, sulfide ore' (kilogram, RoW, None)),
  (0.0022070498622634785,
   2.6055106253254823,
   'heat production, at hard coal industrial furnace 1-10MW' (megajoule, RoW, None)),
  (0.001558553242401338,
   0.004548926676238322,
   'blasting' (kilogram, RoW, None))],
 [(0.3981772912812932,
   1.165215399103026,
   'hard coal mine operation and hard coal preparation' (kilogram, CN, None)),
  (0.29979005949338683,
   2.6055106253254823,
   'heat production, at hard coal industrial furnace 1-10MW' (megajoule, RoW, None)),
  (0.2502559069751329,
   0.18504371146656087,
   'electricity production, hard coal' (kilowatt hour, CN-NM, None)),
  (0.2115533277758135,
   0.014451

In [102]:
#filter the LCIA method again (only keep lvl2 part shown in the contribution analysis table) 

list_methods = []

for method in filtered_key:
    list_methods.append(method[2])   

print(list_methods)

['acidification potential (AP)', 'global warming potential (GWP100)', 'ecotoxicity: freshwater', 'eutrophication potential', 'ozone depletion potential (ODP)']


In [103]:
#list all units of the LCIA method (shown in the contribution analysis table)

list_units = []

for method in filtered_key:
    key = tuple(method)   # Update key for each iteration
    unit = bw.methods.get(key, {'unit': 'unknown unit'}).get('unit')
    list_units.append(unit)

print(list_units)

['kg SO2-Eq', 'kg CO2-Eq', 'CTUe', 'kg N-Eq', 'kg CFC-11-Eq']


In [104]:
#organize the results of contribution analysis as pd.Dataframe

flat_data = [(score, norm_score, process) for sublist in lca_contribution_results for score, norm_score, process in sublist]
ca_result1 = pd.DataFrame(flat_data, columns=['Contribution Score', 'Normalized Contribution', 'Process'])

lcia_methods_column = [method for method in list_methods for _ in range(i)]
ca_result1['LCIA methods'] = lcia_methods_column

lcia_units_column = [method for method in list_units for _ in range(i)]
ca_result1['units'] = lcia_units_column

ca_result1

,Contribution Score,Normalized Contribution,Process,LCIA methods,units
0,2.013516e-02,0.286377,"[comment, classifications, activity type, acti...",acidification potential (AP),kg SO2-Eq
1,2.818746e-03,0.005610,"[comment, classifications, activity type, acti...",acidification potential (AP),kg SO2-Eq
2,2.521855e-03,0.005500,"[comment, classifications, activity type, acti...",acidification potential (AP),kg SO2-Eq
3,2.207050e-03,2.605511,"[comment, classifications, activity type, acti...",acidification potential (AP),kg SO2-Eq
4,1.558553e-03,0.004549,"[comment, classifications, activity type, acti...",acidification potential (AP),kg SO2-Eq
5,3.981773e-01,1.165215,"[comment, classifications, activity type, acti...",global warming potential (GWP100),kg CO2-Eq
6,2.997901e-01,2.605511,"[comment, classifications, activity type, acti...",global warming potential (GWP100),kg CO2-Eq
7,2.502559e-01,0.185044,"[comment, classifications, activity type, acti...",global warming potential (GWP100),kg CO2-Eq
8,2.115533e-01,0.014451,"[comment, classifications, activity type, acti...",global warming potential (GWP100),kg CO2-Eq
9,2.084599e-01,0.149776,"[comment, classifications, activity type, acti...",global warming potential (GWP100),kg CO2-Eq


In [105]:
#export LCA and contribution analysis results to an excel file

excel_file_path = 'C:/Users/89751/OneDrive/桌面/Book1.xlsx'   #create a new Excel file and load the address here

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:

    # Write the first dataFrame to a sheet named 'overall_lca_result'
    Overall_LCA_result.to_excel(writer, sheet_name='overall_lca_result', index=False)
    
    # Write the second DataFrame to a sheet named 'lca_result_activity1'
    LCA_result1.to_excel(writer, sheet_name='lca_result_activity1', index=False)   #update sheet name here

    # Write the third DataFrame (df2) to a sheet named 'contribution_analysis_activity1'
    ca_result1.to_excel(writer, sheet_name='contribution_analysis_activity1', index=False)   #update sheet name here